# LFMC Projection - Mapping Models
Train Nowcasting and 3-month projection models to create LFMC maps

In [1]:
import os
import json
import numpy as np
import pandas as pd

import initialise
import common
from modelling_functions import create_models, run_experiment
from architecture_projection import model_params
from model_parameters import ExperimentParams

## Directories and Input files
Change these settings as required
- `input_dir`: Directory containing the data extracted from GEE and Globe-LFMC, the outputs from running the `Extract DEM Data.ipynb` and `Extract MODIS Data.ipynb` notebooks.
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [2]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_730days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_730days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_730days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

If 'tests' is 'falsy' then a single test will be run using the parameters in the main model_params dictionary.

Other settings are:
- layerTypes: specifies which layers to include in the model
- Layer parameters should be specified as a list. The first entry in the list will be used for the first layer, etc.
- If the experiment includes changes to the layers, all non-default layer parameters need to be included. The parameters that are kept constant can be specified by including a key for the layer type in the experiment dictionary, and the value set to a dictionary of the constant parameters.

Model_parameters that cannot be changed in tests are:
- \*Filename
- \*Channels
- targetColumn

Example of setting layer parameters:
```
{'name': 'Filters',
 'description': 'Test effect of different filter sizes on conv layers',
 'tests': [{'conv': {'filters': [32, 32, 32]}},
           {'conv': {'filters': [8, 8, 8]}},
           {'conv': {'filters': [32, 8, 8]}},
           {'conv': {'filters': [8, 32, 8]}},
           {'conv': {'filters': [8, 8, 32]}},
           {'conv': {'filters': [8, 16, 32]}},
           {'conv': {'filters': [32, 16, 8]}}],
 'conv': {'numLayers': 3, 'poolSize': [2, 3, 4]},
 'restart': 0}
```

In [3]:
experiment = ExperimentParams({
    'name': 'final_models',
    'description': 'Nowcasting and 3-month projection models for the LFMC maps',
    'tests': [
        {'testName': 'Nowcasting',
         'inputs': {'modis': {'start': -365, 'end': 0},
                    'prism': {'start': -365, 'end': 0}}},
        {'testName': '3-months lead time',
         'inputs': {'modis': {'start': -275, 'end': 90},
                    'prism': {'start': -275, 'end': 90}}},
    ],
    'restart': None,
})
experiment

{'name': 'final_models',
 'description': 'Nowcasting and 3-month projection models for the LFMC maps',
 'blocks': {},
 'tests': [{'testName': 'Nowcasting',
   'inputs': {'modis': {'start': -365, 'end': 0},
    'prism': {'start': -365, 'end': 0}}},
  {'testName': '3-months lead time',
   'inputs': {'modis': {'start': -275, 'end': 90},
    'prism': {'start': -275, 'end': 90}}}],
 'testNames': [],
 'restart': None,
 'rerun': [],
 'resumeAllTests': False}

## Set up model parameters
Set up and customise the model parameters. Leave all parameters as set here to run Scenario A. To find out more about any parameter, run `model_params.help('<parameter>')` after running this cell to create the ModelParams object.

In [4]:
# Customize model parameters
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['samplesFile'] = aux_csv
model_params['modelRuns'] = common.ENSEMBLE_SIZE
model_params['tempDir'] = common.TEMP_DIR
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])
model_params['derivedModels'] = common.DERIVED_MODELS
model_params['seedList'] = [
    566, 451, 795, 237, 788, 185, 397, 530, 758, 633,
    914, 326, 334, 366, 336, 413, 111, 599, 416, 230,
]

# Exclude 2018 data from training samples to ensure we don't use any data from the 3-month lead time
model_params['yearColumn'] = 'Sampling year'
model_params['splitMethod'] = 'byYear'
model_params['splitYear'] = 2018

model_params['saveModels'] = True
model_params['gpuDevice'] = 0

model_params

{'modelName': 'final_models',
 'description': 'Nowcasting and 3-month projection models for the LFMC maps',
 'modelClass': 'LfmcTempCnn',
 'modelDir': 'G:\\My Drive\\LFMC Data\\LFMC_projections\\Models\\final_models',
 'tempDir': 'C:\\Temp\\LFMC',
 'diagnostics': False,
 'dataSources': ['modis', 'prism', 'aux'],
 'restartRun': None,
 'derivedModels': {'merge10': {'type': 'merge', 'models': 10}},
 'saveModels': True,
 'saveTrain': None,
 'saveValidation': True,
 'plotModel': True,
 'multiSamples': None,
 'deduplicate': False,
 'randomSeed': 1234,
 'modelSeed': 1234,
 'modelRuns': 20,
 'resplit': False,
 'seedList': [566,
  451,
  795,
  237,
  788,
  185,
  397,
  530,
  758,
  633,
  914,
  326,
  334,
  366,
  336,
  413,
  111,
  599,
  416,
  230],
 'maxWorkers': 1,
 'deterministic': False,
 'gpuDevice': 0,
 'gpuList': [],
 'gpuMemory': 0,
 'inputs': {},
 'samplesFile': 'G:\\My Drive\\LFMC Data\\LFMC_projections\\Datasets\\samples_730days.csv',
 'samplesFilter': None,
 'auxColumns':

In [5]:
model_params.add_input('modis', {'filename': modis_csv, 'channels': 7})
model_params.add_input('prism', {'filename': prism_csv, 'channels': 7})
model_params['inputs']

{'modis': {'filename': 'G:\\My Drive\\LFMC Data\\LFMC_projections\\Datasets\\modis_730days.csv',
  'normalise': {'method': 'minMax', 'percentiles': 2},
  'channels': 7,
  'includeChannels': [],
  'start': None,
  'end': None},
 'prism': {'filename': 'G:\\My Drive\\LFMC Data\\LFMC_projections\\Datasets\\prism_730days.csv',
  'normalise': {'method': 'minMax', 'percentiles': 2},
  'channels': 7,
  'includeChannels': [],
  'start': None,
  'end': None}}

## Build and run the models
Builds and trains the LFMC models.

All models, predictions, evaluation statistics, and plots of test results are saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [6]:
def is_experiment():
    try:
        return bool(experiment['tests'])
    except:
        return False

In [7]:
if is_experiment():
    models = run_experiment(experiment, model_params)
else:
    print('Running a single test')
    models = [create_models(model_params)]
for model in models:
    display(getattr(model, 'all_stats', None))

Experiment final_models - Nowcasting and 3-month projection models for the LFMC maps

Reading samples file G:\My Drive\LFMC Data\LFMC_projections\Datasets\samples_730days.csv
Reading modis file G:\My Drive\LFMC Data\LFMC_projections\Datasets\modis_730days.csv
Reading prism file G:\My Drive\LFMC Data\LFMC_projections\Datasets\prism_730days.csv

----------------------------------------------------------------------

Test 0: Nowcasting - {'inputs': {'modis': {'start': -365, 'end': 0}, 'prism': {'start': -365, 'end': 0}}}

Setting target to "LFMC value"
Auxiliary columns: ['Long_sin', 'Long_cos', 'Lat_norm']
One-hot encoded columns: ['Czone3']
Modis input shape: (66411, 730, 7)
Prepared modis shape: (66411, 365, 7)
Prism input shape: (66411, 730, 7)
Prepared prism shape: (66411, 365, 7)
Prepared aux shape: (66411, 18)

----------------------------------------------------------------------

Test 1: 3-months lead time - {'inputs': {'modis': {'start': -275, 'end': 90}, 'prism': {'start': -275

None

None